In [1]:
import pandas as pd
import numpy as np
import datetime
import os
print(datetime.datetime.now())
os.getcwd()

2020-01-31 16:20:44.871224


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/BR_audience_furniture_browser'

In [2]:
df_test=pd.read_csv("./BL_furniture_browser_test_group_JL_+str(datetime.datetime.now().date()).csv",dtype=str)
df_control=pd.read_csv("./BL_furniture_browser_control_group_JL_+str(datetime.datetime.now().date()).csv",dtype=str)

print(df_test.shape,df_control.shape)

df_test.head(2)

(489861, 5) (54430, 5)


,index,customersummary_mastercustomerid,Email,customer_id_hashed,execution_id
0,0,KFK_0_8017815,ad5f11c041c89839c09204f9ce34f5d97ea8ddaa81f1f0...,83228bfe2f6d62bb0218751ceb63aa8551e9fd89a999f1...,1578944138509
1,2,KFK_0_37564566,c612188e441b8fef2e5eed0c918fec1d12e23f801b9e69...,94245ca46539f480c2490a5e3373f237dbc455094f85b7...,1578944138509


In [3]:
df_group_1=df_test[['customer_id_hashed']].drop_duplicates()
df_group_1['group']='test'
df_group_2=df_control[['customer_id_hashed']].drop_duplicates()
df_group_2['group']='control'

df_group=df_group_1.append(df_group_2)
print(df_group.shape,df_group['customer_id_hashed'].nunique())

df_total_id=df_group.groupby(['group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"total_ids"})

(544291, 2) 544291


In [4]:
df_total_id

,group,total_ids
0,control,54430
1,test,489861


In [5]:
df_sales_by_day_1=pd.read_table("/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-18/MediaStormDailySales20200121-111749-649.txt",
                                sep="|",dtype=str)
df_sales_by_day_2=pd.read_table("/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-25/MediaStormDailySales20200128-111758-074.txt",
                                sep="|",dtype=str)
df_sales_by_day=df_sales_by_day_1.append(df_sales_by_day_2)

del df_sales_by_day_1
del df_sales_by_day_2

In [6]:
df_sales_by_day['item_transaction_amt']=df_sales_by_day['item_transaction_amt'].astype(float)
df_sales_by_day=df_sales_by_day[pd.notnull(df_sales_by_day['customer_id_hashed'])]
df_sales_by_day=pd.merge(df_sales_by_day,df_group,on="customer_id_hashed",how="left")
df_sales_by_day['group']=df_sales_by_day['group'].fillna('others')

In [7]:
df_taxonomy_div=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",dtype=str,sep="|")
df_taxonomy_div=df_taxonomy_div[['division_id','class_code_id','subclass_id']].drop_duplicates()

df_sales_by_day=pd.merge(df_sales_by_day,df_taxonomy_div,on=['class_code_id','subclass_id'],how="left")
# df_sales_by_day['store_type']=np.where(df_sales_by_day['location_id']=="6990","online","instore")

# No Store Type

In [10]:
agg_fun={"customer_id_hashed":"nunique","item_transaction_amt":"sum"}
df_total_sales=df_sales_by_day.groupby(['transaction_dt','group'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_by_day[['group','location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','group'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','group'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_by_day.groupby(['transaction_dt','group','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_by_day[['group','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','group','division_id'])['trans'].sum().to_frame().reset_index()


In [11]:
df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [12]:
pd.options.display.max_columns=99

df_output=pd.merge(df_total,df_division,on=['transaction_dt','group'],how="outer")

df_output=pd.merge(df_total_id,df_output,on="group",how="outer")
df_output=df_output.sort_values(["transaction_dt","group"])
# df_output

# 3 days overall

In [14]:
df_sales_3_campaign_days=df_sales_by_day[df_sales_by_day['transaction_dt'].isin(['2020-01-18','2020-01-19','2020-01-20'])]
df_sales_3_campaign_days=df_sales_3_campaign_days.rename(columns={"transaction_dt":"date_kept_to_dedup"})

df_sales_3_campaign_days['transaction_dt']='total_3_days_Jan_18-20'

df_total_sales=df_sales_3_campaign_days.groupby(['transaction_dt','group'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_3_campaign_days[['group','location_id','transaction_dt','transaction_id','customer_id_hashed','date_kept_to_dedup']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','group'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','group'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_3_campaign_days.groupby(['transaction_dt','group','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_3_campaign_days[['group','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','date_kept_to_dedup']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','group','division_id'])['trans'].sum().to_frame().reset_index()



In [15]:
# 3 days overall

df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [16]:
df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [17]:
df_output_2=pd.merge(df_total,df_division,on=['transaction_dt','group'],how="outer")

df_output_2=pd.merge(df_total_id,df_output_2,on="group",how="outer")
df_output_2=df_output_2.sort_values(["transaction_dt","group"])

df_output=df_output.append(df_output_2)

In [18]:
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")

writer=pd.ExcelWriter("./BL_performance_of_furniture_browser_groups_updated_3_Campaign_days_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"view_no_store_type",index=False)
df_division_name.to_excel(writer,"divison_name",index=False)
writer.save()